Making Shor's algo scalable (Starting with preexisiting hard coded Shors from textbook)
    1️ Parameterize the problem (remove small-N assumptions)
    What this means
    N is an input, not baked into the circuit. All register sizes come from n = ceil(log2(N))
    Why it matters
    Scalability is about growth with log N. Hardcoding N kills scalability immediately

    2️ Replace hardcoded U with general modular exponentiation
    What this means
    No hand-built modular multiply gates. Modular exponentiation is built from:
        reversible adders, modular multiplication, repeated squaring
    Why it matters
    This is the heart of Shor’s
    This is where “toy demo” → “real algorithm” happens

    3️ Use qubit-efficient phase estimation
    What this means

Replace full QFT with iterative / semiclassical QPE

Why it matters

Reduces qubit requirements

Keeps algorithm executable on near-term hardware

Still mathematically identical

4️⃣ Separate algorithm from hardware execution

What this means

One layer builds circuits

Another layer:

runs on simulators (small N)

runs on hardware (tiny N)

estimates resources (large N)

Why it matters

Same algorithm works today and in the future

Backend changes, algorithm doesn’t

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction

ModuleNotFoundError: No module named 'pandas'

In [4]:
def controlled_modular_multiply(a, N, n):
    """Placeholder for modular multiplication by a mod N on an n-qubit register.

    In a full implementation, this would be a reversible modular
    multiplication circuit built from adders and modular reduction.
    We keep it as a stub so the modular-exponentiation structure is
    explicit and scalable with log(N).
    """
    qc = QuantumCircuit(n)
    # Placeholder: identity. Replace with reversible modular multiplication.
    qc.barrier()
    gate = qc.to_gate()
    gate.name = f"x{a} mod {N}"
    return gate.control()


def a2jmodN(a, j, N):
    """Compute a^(2^j) (mod N) by repeated squaring (classical helper).

    This mirrors the structure used in the quantum circuit: each counting
    qubit controls a multiplication by a^(2^j) mod N.
    """
    for _ in range(j):
        a = (a * a) % N
    return a


def c_amodN(a, power, N, n):
    """Controlled multiplication by a^(2^power) mod N.

    Modular exponentiation is the heart of Shor's algorithm. The
    repeated-squaring structure (powers of two) is what makes the
    circuit scale with log(N) instead of being hardcoded for a
    particular small N.
    """
    a_power = a2jmodN(a, power, N)
    return controlled_modular_multiply(a_power, N, n)


# Specify variables
N = 15
n = int(np.ceil(np.log2(N)))  # register size scales with log2(N)
N_COUNT = 2 * n  # typical choice for phase estimation
a = 7


def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ
    """
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFTdagger"
    return qc


In [5]:
# Create QuantumCircuit with N_COUNT counting qubits
# plus n qubits for modular multiplication
qc = QuantumCircuit(N_COUNT + n, N_COUNT)

# Initialize counting qubits
# in state |+>
for q in range(N_COUNT):
    qc.h(q)

# And auxiliary register in state |1>
qc.x(N_COUNT)

# Modular exponentiation: build |x>|1> -> |x>|a^x mod N>
# using repeated squaring. Each counting qubit controls
# multiplication by a^(2^q) mod N.
# This replaces hardcoded small-N gates and scales with log(N).
for q in range(N_COUNT):
    qc.append(c_amodN(a, q, N, n),
             [q] + [i+N_COUNT for i in range(n)])

# Do inverse-QFT
qc.append(qft_dagger(N_COUNT), range(N_COUNT))

# Measure circuit
qc.measure(range(N_COUNT), range(N_COUNT))
qc.draw(fold=-1)  # -1 means 'do not fold'


QiskitError: 'One or more instructions cannot be converted to a gate. "barrier" is not a gate instruction'

In [6]:

aer_sim = Aer.get_backend('aer_simulator')
t_qc = transpile(qc, aer_sim)
counts = aer_sim.run(t_qc).result().get_counts()
plot_histogram(counts)
rows, measured_phases = [], []
for output in counts:
    decimal = int(output, 2)  # Convert (base 2) string to decimal
    phase = decimal/(2**N_COUNT)  # Find corresponding eigenvalue
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append([f"{output}(bin) = {decimal:>3}(dec)",
                 f"{decimal}/{2**N_COUNT} = {phase:.2f}"])
# Print the rows in a table
headers=["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)


NameError: name 'Aer' is not defined

In [7]:

Fraction(0.666)

# Get fraction that most closely resembles 0.666
# with denominator < 15
Fraction(0.666).limit_denominator(15)
rows = []
for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(15)
    rows.append([phase,
                 f"{frac.numerator}/{frac.denominator}",
                 frac.denominator])
# Print as a table
headers=["Phase", "Fraction", "Guess for r"]
df = pd.DataFrame(rows, columns=headers)
print(df)

NameError: name 'Fraction' is not defined

In [8]:
a2jmodN(7, 2049, 53)

N = 15
n = int(np.ceil(np.log2(N)))

np.random.seed(1) # This is to make sure we get reproduceable results
a = randint(2, 15)
print(a)

from math import gcd # greatest common divisor
gcd(a, N)
def qpe_amodN(a, N):
    """Performs quantum phase estimation on the operation a*r mod N.
    Args:
        a (int): This is 'a' in a*r mod N
        N (int): Composite to factor
    Returns:
        float: Estimate of the phase
    """
    n = int(np.ceil(np.log2(N)))
    N_COUNT = 2 * n
    qc = QuantumCircuit(n + N_COUNT, N_COUNT)
    for q in range(N_COUNT):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(N_COUNT) # And auxiliary register in state |1>
    for q in range(N_COUNT): # Do controlled-U operations
        qc.append(c_amodN(a, q, N, n),
                 [q] + [i+N_COUNT for i in range(n)])
    qc.append(qft_dagger(N_COUNT), range(N_COUNT)) # Do inverse-QFT
    qc.measure(range(N_COUNT), range(N_COUNT))
    # Simulate Results
    aer_sim = Aer.get_backend('aer_simulator')
    # `memory=True` tells the backend to save each measurement in a list
    job = aer_sim.run(transpile(qc, aer_sim), shots=1, memory=True)
    readings = job.result().get_memory()
    print("Register Reading: " + readings[0])
    phase = int(readings[0],2)/(2**N_COUNT)
    print(f"Corresponding Phase: {phase}")
    return phase

phase = qpe_amodN(a, N) # Phase = s/r
Fraction(phase).limit_denominator(15)

frac = Fraction(phase).limit_denominator(15)
s, r = frac.numerator, frac.denominator
print(r)
guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
print(guesses)


7


QiskitError: 'One or more instructions cannot be converted to a gate. "barrier" is not a gate instruction'

In [9]:
a = 7
FACTOR_FOUND = False
ATTEMPT = 0
while not FACTOR_FOUND:
    ATTEMPT += 1
    print(f"\nATTEMPT {ATTEMPT}:")
    phase = qpe_amodN(a, N) # Phase = s/r
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    print(f"Result: r = {r}")
    if phase != 0:
        # Guesses for factors are gcd(x^{r/2} +- 1 , N)
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
        print(f"Guessed Factors: {guesses[0]} and {guesses[1]}")
        for guess in guesses:
            if guess not in [1,N] and (N % guess) == 0:
                # Guess is a factor!
                print(f"*** Non-trivial factor found: {guess} ***")
                FACTOR_FOUND = True


# The cell below repeats the algorithm until at least one factor of 15
# is found
assert (3 in guesses) or (5 in guesses)



ATTEMPT 1:


QiskitError: 'One or more instructions cannot be converted to a gate. "barrier" is not a gate instruction'